# Dog Breed Classifier

In [12]:
import pandas as pd
import numpy as np
#glob allows to recursively retrieve files that match a specified petter.
#The reason we use glob here is to obtain all the files inside the folders in each route
from glob import glob

## The dataset

We have 8,351 dog images and 5,749 human images. 

The cell below uses the glob command to find all the images inside the folders obtained after decompressing the ZIP files. The file names are stored in two lists, ```dog_images``` and ```people_images```.

In [24]:
dog_images=glob("images\dogs\*\*\*\*")
people_images=glob("images\people\*\*")

print("We have {:,} dog images in total (train+dev+test)".format(len(dog_images)))
print("We have {:,} human images in total (train+dev+test)".format(len(people_images)))

We have 8,351 dog images in total (train+dev+test)
We have 5,749 human images in total (train+dev+test)


## Human detection

The **OpenCV** library will be used to detect human faces. This is an open-source library widely used for computer vision and image processing.

The algorithm to be implemented is a Haar feature-based cascade classifier. This model was first proposed in 2001 by Viola and Jones in their [research paper](https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/viola-cvpr-01.pdf) "Rapid Object Detection using a Boosted Cascade of Simple features". This is a very efficient and highly accurate algorithm, extensively used even in the current deep learning era.

In simple, the algorithm uses what is known as Haar features to detect attributes in the image. These features have a similar structure to kernel matrices on CNN, with the difference that the values inside the matrix are assigned a priori (not estimated) and take the values between 0 (white) and 1 (black).

In the publication, the authors describe the 3 main contributions of their work:
* Integral image.
* Feature selection with AdaBoost.
* Combining complex classifiers in a cascade structure.

The three findings listed above contribute to the efficiency and high detection rate achieved my the technique.

---

OpenCV already has some pretrained detectors that can be found in the following [link](https://github.com/opencv/opencv/tree/master/data/haarcascades). Some of the trained classifiers include eye-detection, smile-detection, russiand plate number, and more.

For the purpose of detection faces, I will use ```haarcascade_frontalface_alt.xml``` 

In [1]:
#import OpenCV
import cv2

In [2]:
cv2

<module 'cv2' from 'C:\\Users\\USUARIO\\anaconda3\\lib\\site-packages\\cv2.cp38-win_amd64.pyd'>